In [1]:
train_nq_path = 'C:\\Users\\Matheus\\Desktop\\Pós Graduação\\NLP\\Projeto\\Datasets\\NATURAL_QUESTIONS\simplified-nq-train.jsonl'


In [2]:
import jsonlines
import pdb
import re

In [3]:
class SimplifyNQ():
    def __init__(self, nq_example):
        
        self.nq_example = nq_example
        self.example_id = nq_example['example_id']
        self.document_url = nq_example['document_url']
        self.question_text = nq_example['question_text']
        self.document_text = nq_example['document_text']
        self.annotations = nq_example['annotations']
        
          

    
    def text_filter(self,original_text):
        text = re.sub('{.*?}', '', original_text) #Remoção das tags de color:{#000000}
        text = re.sub('<.*?>', '', text) #Remoção das tags de espaço ou qualquer outra que aparecer
        text = re.sub('/\r\n|\n|\r|', '',text);#Remoção de line breaks por \n e \r
        text = re.sub(' +', ' ',text) #Transformação de multiplos espaços em um único espaço
        text = text.strip(' ')
        return text
    
        
        
        
    def define_newstart_newend_long_answer(self,type_answer='long_answer'):
        
        
        start_token = self.annotations[0][type_answer]['start_token']
        end_token = self.annotations[0][type_answer]['end_token']
        
        new_start_token_list = self.text_filter(" ".join(self.document_text.split(" ")[:start_token])).split(" ")
        new_start_token = 0 if (new_start_token_list[0]=='' and len(new_start_token_list)==1) else len(new_start_token_list)
        
        
        
        new_end_token = len(self.text_filter(" ".join(self.document_text.split(" ")[:end_token])).split(" "))
        
        
        
        return new_start_token, new_end_token
        
    
    def get_long_answer(self):
        
        long_answer_dict = {}
        start_token = self.annotations[0]['long_answer']['start_token']
        end_token = self.annotations[0]['long_answer']['end_token']
        long_answer = " ".join(self.document_text.split(" ")[start_token:end_token])
        long_answer = self.text_filter(long_answer)
        new_start_token, new_end_token = self.define_newstart_newend_long_answer()
        
        
#         long_answer_dict['long_answer'] = long_answer
        long_answer_dict['start_token'] = new_start_token
        long_answer_dict['end_token'] = new_end_token
        
        return long_answer_dict
    
    
    
    def define_newstart_newend_short_answer(self,i,type_answer='short_answers'):
        
        
        start_token = self.annotations[0][type_answer][i]['start_token']
        end_token = self.annotations[0][type_answer][i]['end_token']
        
        new_start_token_list = self.text_filter(" ".join(self.document_text.split(" ")[:start_token])).split(" ")
        new_start_token = 0 if (new_start_token_list[0]=='' and len(new_start_token_list)==1) else len(new_start_token_list)
        new_end_token = len(self.text_filter(" ".join(self.document_text.split(" ")[:end_token])).split(" "))
        
        return new_start_token, new_end_token
    
    
    def get_short_answers(self):
        short_answers = []
        for i,short_answer in enumerate(self.annotations[0]['short_answers']):
            short_answer_dict = {}
            start_token = short_answer['start_token']
            end_token = short_answer['end_token']
            short_answer = " ".join(self.document_text.split(" ")[start_token:end_token])
            short_answer = self.text_filter(short_answer)
            
            
            
            new_start, new_end = self.define_newstart_newend_short_answer(i)
#             short_answer_dict['short_answer'] = short_answer
            short_answer_dict['start_token'] = new_start
            short_answer_dict['end_token'] = new_end
            
            short_answers.append(short_answer_dict)
            
            
        
        return short_answers



    def get_yes_no_answer(self):
        return self.annotations[0]['yes_no_answer']
         
    
    
    def get_nq_simplifyed(self):
        nq_simplified =   {
            "example_id": self.example_id ,
            "document_url": self.document_url,
            "question_text": self.text_filter(self.question_text),
            "document_text": self.text_filter(self.document_text),
            "annotations": [{
              "long_answer": self.get_long_answer(),
              "short_answers": self.get_short_answers(),
              "yes_no_answer": self.get_yes_no_answer(),
            }]
        }
        return nq_simplified

In [10]:
dummy_example = {
        "example_id": 3902,
        "document_url": "http://wikipedia.org/en/strings",
        "question_text": "<h1>what is a string</h1>",
        "document_text": "<P> A string is a list of characters in order . </P>",
        "annotations": [{
          "long_answer": { "start_token": 0, "end_token": 12 },
          "short_answers": [{ "start_token": 5, "end_token": 8 }],
          "yes_no_answer": "NONE",
        }],
        "long_answer_candidates": [
          {"start_token": 0, "end_token": 12, "top_level": True}
        ]
      }

In [11]:
simple_nq = SimplifyNQ(dummy_example)
simple_nq.get_nq_simplifyed()

{'example_id': 3902,
 'document_url': 'http://wikipedia.org/en/strings',
 'question_text': 'what is a string',
 'document_text': 'A string is a list of characters in order .',
 'annotations': [{'long_answer': {'start_token': 0, 'end_token': 10},
   'short_answers': [{'start_token': 4, 'end_token': 7}],
   'yes_no_answer': 'NONE'}]}

In [6]:
def save_json_simple(num_elements):
    with jsonlines.open(train_nq_path) as reader:
        with jsonlines.open('clean_nq_format.jsonl', mode='w') as writer:
            for i, obj in enumerate(reader):
                simple_nq = SimplifyNQ(obj)
                writer.write(simple_nq.get_nq_simplifyed())
                if i == num_elements:
                    break

In [7]:
save_json_simple(1000)